# PROJET ANNUEL Véhicule autonome et Deep Learning
## PRONOST Sacha, NEVEU Thomas, VASSE Thomas, BERNEAUD Noah

In [ ]:
import flash
from flash.core.data.utils import download_data
from flash.image import SemanticSegmentation, SemanticSegmentationData

import matplotlib.pyplot as plt
import numpy as np
import torch
import PIL
import os
import torch
import torch.utils.data.dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.model_selection import train_test_split
import torchmetrics

# Préparation des données

### Les données récupérées sont sur le [lien](https://npm3d.fr/kitti-carla) suivant:
 https://npm3d.fr/kitti-carla
 #### (Il faut les traiter à l'aide du fichier 'convert_image_ss' au préalable)
### Les anciennes données sont sur le [lien](https://github.com/ongchinkiat/LyftPerceptionChallenge/releases/download/v0.1/carla-capture-20180513A.zip) suivant:
 https://github.com/ongchinkiat/LyftPerceptionChallenge/releases/download/v0.1/carla-capture-20180513A.zip

In [ ]:
# Séparation de la donnée
X = []
for filename in os.listdir('data/Town01/generated/images_rgb/'):
    X.append('data/Town01/generated/images_rgb/'+filename)
y = []
for filename in os.listdir('data/Town01/generated/image_ss_new/'):
    y.append('data/Town01/generated/image_ss_new/'+filename)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
datamodule = SemanticSegmentationData.from_files(
    train_files=X_train,
    train_targets=y_train,
    val_split=0.1,
    transform_kwargs=dict(image_size=(256, 256)),
    num_classes=23,
    batch_size=4,
)

# Vérification des images d'apprentissage

In [ ]:
data = next(iter(datamodule.train_dataloader()))

In [ ]:
print(data.keys())

In [ ]:
im = data['input'][0]
y = data['target'][0]

In [ ]:
plt.imshow(im.numpy().transpose(1,2,0))
plt.show()

In [ ]:
plt.imshow(y)
plt.show()


# Apprentissage

In [ ]:
model = SemanticSegmentation(
    #backbone="mobilenetv3_large_100",
    backbone="resnet50",
    head="fpn",
    num_classes=datamodule.num_classes,
)

In [ ]:
trainer = flash.Trainer(max_epochs=3, accelerator='tpu')#torch.cuda.device_count())
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

In [ ]:
# Sauvegarde du modèle
trainer.save_checkpoint("model/modelMobileNetWithTestSplit.pt")

In [5]:
#Si le modèle est déjà crée, l'ouvrir et le charger
model = model.load_from_checkpoint('model/modelMobileNetWithTestSplit.pt')
trainer = flash.Trainer(max_epochs=3, gpus=0)

Using 'mobilenetv3_large_100' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).
Using 'fpn' provided by qubvel/segmentation_models.pytorch (https://github.com/qubvel/segmentation_models.pytorch).
GPU available: False, used: False
TPU available: False, using: 0 TPU cores


# Teste de quelques images

In [ ]:
datamodule = SemanticSegmentationData.from_files(
    predict_files=[
        "data/Town01/generated/images_rgb/2.png"
    ],
    batch_size=3,
)
predictions = trainer.predict(model, datamodule=datamodule)


In [ ]:
in_im_test = predictions[0][0]['input']
out_im_test = predictions[0][0]['preds']

in_im_test=in_im_test.numpy().transpose(1,2,0)
in_im_test=(in_im_test-np.min(in_im_test))/(np.max(in_im_test)-np.min(in_im_test))

out_im_test = torch.argmax(out_im_test,0)

#plt.imshow(in_im_test[:,:,0],cmap='gray')
plt.imshow(in_im_test)
plt.imshow(out_im_test,alpha=0.3)
plt.show()

# Verification des labels

In [ ]:
datamodule = SemanticSegmentationData.from_files(
    test_files=[
        "data/Town01/generated/images_rgb/2.png",
        "data/Town01/generated/images_rgb/4.png",
        "data/Town01/generated/images_rgb/6.png",
    ],
    test_targets=[
        "data/Town01/generated/image_ss_new/2.png",
        "data/Town01/generated/image_ss_new/4.png",
        "data/Town01/generated/image_ss_new/6.png",
    ],
    transform_kwargs=dict(image_size=(256, 256)),
    num_classes=23,    
    batch_size=3,
)

In [ ]:
data = next(iter(datamodule.test_dataloader()))
im = data['input'][0]
y = data['target'][0]

plt.imshow(im.numpy().transpose(1,2,0))
plt.show()

plt.imshow(y)
plt.show()

# Evaluation du modèle

In [6]:
# Préparation de la donnée
X_test = X_test[:100]
y_test = y_test[:100]
print(len(X_test))

100


In [7]:
modulePred = SemanticSegmentationData.from_files(
        predict_files=X_test,
        batch_size=1,
)
predictions = trainer.predict(model, datamodule=modulePred)

c:\Users\sacha\anaconda3\envs\deepLearningVehicule\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, predict dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Predicting: 0it [00:00, ?it/s]

In [8]:
accuracy = 0
accuracyMetric = torchmetrics.Accuracy(multiclass=True,num_classes=23)
moduleTest = SemanticSegmentationData.from_files(
        test_files=X_test,
        test_targets=y_test,
        batch_size=1,
    )
toPrint = [0,10,20,30,40,50,60,70,80,90]
for i in range(100):
    prediction = torch.argmax(predictions[i][0]['preds'], 0).flatten()
    true = moduleTest.test_dataloader().dataset[i]['target'].flatten()

    #Conversion de ce qui ce trouve dans le tensor predictions et true en int
    prediction = prediction.numpy().astype(int)
    true = true.numpy().astype(int)
    #Converstion de predictions et true en tensor
    prediction = torch.from_numpy(prediction)
    true = torch.from_numpy(true)
    #print(true.flatten().shape)
    accuracy += accuracyMetric(prediction, true)
    if(i in toPrint):
        print("Accuracy actuelle: ",accuracyMetric(prediction, true))
        print("Accuracy moyenne: ",accuracy/(i+1))
print(accuracy/len(X_test))

Accuracy actuelle:  tensor(0.9234)
Accuracy moyenne:  tensor(0.9234)
Accuracy actuelle:  tensor(0.9080)
Accuracy moyenne:  tensor(0.9140)
Accuracy actuelle:  tensor(0.9097)
Accuracy moyenne:  tensor(0.9125)
Accuracy actuelle:  tensor(0.9055)
Accuracy moyenne:  tensor(0.9147)
Accuracy actuelle:  tensor(0.9032)
Accuracy moyenne:  tensor(0.9099)
Accuracy actuelle:  tensor(0.8659)
Accuracy moyenne:  tensor(0.9082)
Accuracy actuelle:  tensor(0.9141)
Accuracy moyenne:  tensor(0.9083)
Accuracy actuelle:  tensor(0.9466)
Accuracy moyenne:  tensor(0.9094)
Accuracy actuelle:  tensor(0.9140)
Accuracy moyenne:  tensor(0.9093)
Accuracy actuelle:  tensor(0.9300)
Accuracy moyenne:  tensor(0.9101)
tensor(0.9099)
